In [1]:
import numpy as np
import tensorflow as tf

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#构建一个简单的分类问题数据集，feature为一个1x5的随机向量，label取值为0或1

#生成10个随机样本，其中一半样本label为0，另一半为1
n = 10
size = (n, 5)
x_data = np.random.randint(0, 10, size=size)
y1_data = np.ones((n//2, 1), int)
y2_data = np.zeros((n//2, 1), int)
#y_data = np.append(y1_data, y2_data)
y_data = np.vstack((y1_data, y2_data))
np.random.shuffle(y_data)
xy_data = np.hstack((x_data,y_data))
x_data = x_data.tolist()
print(xy_data)

[[2 0 0 5 8 1]
 [8 3 7 5 1 1]
 [3 5 7 8 7 1]
 [5 2 7 9 9 0]
 [0 1 0 3 0 0]
 [0 3 4 2 5 0]
 [4 8 8 3 8 1]
 [3 5 2 7 7 0]
 [0 4 7 7 3 1]
 [5 0 2 4 9 0]]


In [3]:
tfrecord_path = 'data.record'
writer = tf.python_io.TFRecordWriter(tfrecord_path)
for i in range(n):
    #读入的数据需要先转化为list
    sample = x_data[i] 
    label = int(y_data[i])
    example = tf.train.Example(features=tf.train.Features(feature={
        'sample':
            tf.train.Feature(int64_list=tf.train.Int64List(value=sample)),
        'label':
            tf.train.Feature(int64_list=tf.train.Int64List(value=[label])),
    }))
    writer.write(example.SerializeToString())
    print(example)
    print(example.SerializeToString())
writer.close()

features {
  feature {
    key: "label"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "sample"
    value {
      int64_list {
        value: 2
        value: 0
        value: 0
        value: 5
        value: 8
      }
    }
  }
}

b'\n%\n\x13\n\x06sample\x12\t\x1a\x07\n\x05\x02\x00\x00\x05\x08\n\x0e\n\x05label\x12\x05\x1a\x03\n\x01\x01'
features {
  feature {
    key: "label"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "sample"
    value {
      int64_list {
        value: 8
        value: 3
        value: 7
        value: 5
        value: 1
      }
    }
  }
}

b'\n%\n\x13\n\x06sample\x12\t\x1a\x07\n\x05\x08\x03\x07\x05\x01\n\x0e\n\x05label\x12\x05\x1a\x03\n\x01\x01'
features {
  feature {
    key: "label"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "sample"
    value {
      int64_list {
        value: 3
        value: 5
        value: 7
        value: 8
   

In [5]:
#读取tfrecord文件
input_filename = 'data.record'
if_enq_many = False

#建立文件名队列
filename_queue = tf.train.string_input_producer([input_filename], num_epochs=3)

# 建立阅读器
reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)

# 根据数据格式建立相对应的读取features
features = {
    'sample': tf.FixedLenFeature([5], tf.int64),# 如果不是标量，一定要在这里说明数组的长度
    'label': tf.FixedLenFeature([], tf.int64)
}
# 解析单个EXAMPLE
Features = tf.parse_single_example(serialized_example, features)
sample = tf.cast(Features['sample'], tf.float32)
label = tf.cast(Features['label'], tf.float32)


sample_single, label_single = tf.train.batch([sample, label],
                                                batch_size=2,
                                                capacity=200,
                                                num_threads=1,
                                                enqueue_many = if_enq_many)
                                                
'''                                                

sample_single, label_single = tf.train.shuffle_batch([sample, label],
                                                 batch_size=2,
                                                 capacity=200,
                                                 min_after_dequeue=100,
                                                 num_threads=1,
                                                 enqueue_many=if_enq_many)
'''
print(sample_single, label_single)

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    #如果tf.train.string_input_producer([tfrecord_path], num_epochs=3)中num_epochs不为空，必须要初始化local变量
    sess.run(tf.local_variables_initializer())
    coord = tf.train.Coordinator()  # 管理线程
    threads = tf.train.start_queue_runners(coord=coord)  # 文件名开始进入文件名队列和内存
    for i in range(n):
        # Queue + tf.parse_single_example()读取tfrecord文件
        X1, Y1 = sess.run([sample_single, label_single])
        print('X1: ', X1, 'Y1: ', Y1) # 这里就可以得到tensor具体的数值
    coord.request_stop()
    coord.join(threads)

Tensor("batch:0", shape=(2, 5), dtype=float32) Tensor("batch:1", shape=(2,), dtype=float32)
X1:  [[2. 0. 0. 5. 8.]
 [8. 3. 7. 5. 1.]] Y1:  [1. 1.]
X1:  [[3. 5. 7. 8. 7.]
 [5. 2. 7. 9. 9.]] Y1:  [1. 0.]
X1:  [[0. 1. 0. 3. 0.]
 [0. 3. 4. 2. 5.]] Y1:  [0. 0.]
X1:  [[4. 8. 8. 3. 8.]
 [3. 5. 2. 7. 7.]] Y1:  [1. 0.]
X1:  [[0. 4. 7. 7. 3.]
 [5. 0. 2. 4. 9.]] Y1:  [1. 0.]
X1:  [[2. 0. 0. 5. 8.]
 [8. 3. 7. 5. 1.]] Y1:  [1. 1.]
X1:  [[3. 5. 7. 8. 7.]
 [5. 2. 7. 9. 9.]] Y1:  [1. 0.]
X1:  [[0. 1. 0. 3. 0.]
 [0. 3. 4. 2. 5.]] Y1:  [0. 0.]
X1:  [[4. 8. 8. 3. 8.]
 [3. 5. 2. 7. 7.]] Y1:  [1. 0.]
X1:  [[0. 4. 7. 7. 3.]
 [5. 0. 2. 4. 9.]] Y1:  [1. 0.]


In [8]:
'''
用tf.parse_example()批量读取数据
args:
      filename_queue: 文件名队列
      shuffle_batch: 是否批量读取数据
'''
# 第一步： 建立文件名队列
input_filename = 'data.record'

filename_queue = tf.train.string_input_producer([input_filename])

reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)

'''
batch = tf.train.shuffle_batch([serialized_example],
                        batch_size=3,
                        capacity=10000,
                        min_after_dequeue=1000,
                        num_threads=1,
                        enqueue_many=False)


'''
batch = tf.train.batch([serialized_example],
                        batch_size=3,
                        capacity=10000,
                        num_threads=1,
                        enqueue_many=False)


features = {
    'sample': tf.FixedLenFeature([5], tf.int64),  # 如果不是标量，一定要在这里说明数组的长度
    'label': tf.FixedLenFeature([], tf.int64)
}

Features = tf.parse_example(batch, features)

samples_parse = tf.cast(Features['sample'], tf.float32)
labels_parse = tf.cast(Features['label'], tf.float32)

print(samples_parse, labels_parse)

init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer()) #需要将string_input_producer也初始化
with tf.Session() as sess:
    sess.run(init)  # 初始化
    coord = tf.train.Coordinator()  # 管理线程
    threads = tf.train.start_queue_runners(coord=coord)  # 文件名开始进入文件名队列和内存
    for i in range(1):
        X2, Y2 = sess.run([samples_parse, labels_parse])
        print('X2: ', X2, 'Y2: ', Y2)

    coord.request_stop()
    coord.join(threads)

Tensor("Cast_8:0", shape=(3, 5), dtype=float32) Tensor("Cast_9:0", shape=(3,), dtype=float32)
X2:  [[2. 0. 0. 5. 8.]
 [8. 3. 7. 5. 1.]
 [3. 5. 7. 8. 7.]] Y2:  [1. 1. 1.]


In [10]:
tfrecord_path = 'data.record'
def _parse_function(example_proto): # 解析函数
    dics = {  
        'sample': tf.FixedLenFeature([5], tf.int64),  # 如果不是标量，一定要在这里说明数组的长度
        'label': tf.FixedLenFeature([], tf.int64)}
    parsed_example = tf.parse_single_example(example_proto, dics)
    parsed_example['sample'] = tf.cast(parsed_example['sample'], tf.float32)
    parsed_example['label'] = tf.cast(parsed_example['label'], tf.float32)

    return parsed_example

def read_dataset(tfrecord_path = tfrecord_path):
    dataset = tf.data.TFRecordDataset(tfrecord_path)
    new_dataset = dataset.map(_parse_function)
    shuffle_dataset = new_dataset.shuffle(buffer_size=20000) # 打乱顺序
    batch_dataset = shuffle_dataset.batch(2) # 按batch数量输出
    prefetch_dataset = batch_dataset.prefetch(2000) # 数据提前进入队列，速度会快很多
    iterator = prefetch_dataset.make_one_shot_iterator()
    next_element = iterator.get_next()
    return next_element

next_element = read_dataset()
init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
with tf.Session() as sess:
    sess.run(init)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord) # 文件名开始进入文件名队列和内存
    for i in range(5):
        print('dataset:', sess.run([next_element['sample'],
                                    next_element['label']]))

    coord.request_stop()
    coord.join(threads)

dataset: [array([[2., 0., 0., 5., 8.],
       [8., 3., 7., 5., 1.]], dtype=float32), array([1., 1.], dtype=float32)]
dataset: [array([[3., 5., 2., 7., 7.],
       [0., 1., 0., 3., 0.]], dtype=float32), array([0., 0.], dtype=float32)]
dataset: [array([[5., 0., 2., 4., 9.],
       [5., 2., 7., 9., 9.]], dtype=float32), array([0., 0.], dtype=float32)]
dataset: [array([[0., 4., 7., 7., 3.],
       [4., 8., 8., 3., 8.]], dtype=float32), array([1., 1.], dtype=float32)]
dataset: [array([[3., 5., 7., 8., 7.],
       [0., 3., 4., 2., 5.]], dtype=float32), array([1., 0.], dtype=float32)]
